# Fit Neg Binomial Variance Component model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial copula for 10 iterations
    - Fix $\beta$ and AR parameters, fit $r$ using Newton for 10 iterations
    - Repeat until convergence

# Block updates

In [8]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

# Random.seed!(1234)
Random.seed!(12345)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [2.0, 8.0, 3.0, 3.0, 3.0]
 [3.0, 8.0, 5.0, 9.0, 4.0]
 [2.0, 12.0, 2.0, 4.0, 2.0]
 [2.0, 5.0, 2.0, 11.0, 3.0]
 [9.0, 1.0, 7.0, 4.0, 3.0]
 [4.0, 3.0, 3.0, 14.0, 2.0]
 [7.0, 1.0, 3.0, 7.0, 2.0]
 [2.0, 5.0, 3.0, 3.0, 8.0]
 [2.0, 1.0, 0.0, 8.0, 6.0]
 [4.0, 4.0, 8.0, 5.0, 6.0]
 [4.0, 2.0, 7.0, 4.0, 0.0]
 [6.0, 6.0, 5.0, 2.0, 2.0]
 [5.0, 3.0, 4.0, 4.0, 7.0]
 ⋮
 [5.0, 1.0, 3.0, 3.0, 7.0]
 [3.0, 1.0, 9.0, 5.0, 6.0]
 [4.0, 1.0, 6.0, 0.0, 8.0]
 [2.0, 7.0, 5.0, 4.0, 6.0]
 [5.0, 2.0, 0.0, 2.0, 4.0]
 [4.0, 9.0, 7.0, 7.0, 4.0]
 [4.0, 7.0, 3.0, 2.0, 2.0]
 [4.0, 2.0, 6.0, 6.0, 8.0]
 [6.0, 1.0, 9.0, 7.0, 3.0]
 [2.0, 4.0, 3.0, 1.0, 1.0]
 [5.0, 4.0, 8.0, 3.0, 4.0]
 [4.0, 3.0, 3.0, 2.0, 12.0]

In [9]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [10]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [1.5037884661509515]
initializing variance components using MM-Algorithm
gcm.Σ = [1.0, 1.0]
initializing variance components using MM-Algorithm
gcm.β = [1.4728457803191453]
gcm.r = [9.875364724278391]


1-element Vector{Float64}:
 9.875364724278391

In [11]:
β_true

1.4552872326068422

In [12]:
GLMCopula.loglikelihood!(gcm, true, true)

-118990.43309562653

In [13]:
# default is Quasi-Newton
@time GLMCopula.fit!(gcm, maxBlockIter=100)

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [1.5037884661509515]
initializing variance components using MM-Algorithm
gcm.Σ = [1.0, 1.0]
initializing variance components using MM-Algorithm
Converging when tol ≤ 1.0e-6 (max block iter = 100)
Block iter 1 r = 11.78, logl = -119004.65, tol = 119004.64713724186
Block iter 2 r = 11.13, logl = -118986.98, tol = 0.00014848610335722697
Block iter 3 r = 10.74, logl = -118980.57, tol = 5.38677622748822e-5
Block iter 4 r = 10.5, logl = -118977.93, tol = 2.217852160694699e-5
Block iter 5 r = 10.34, logl = -118976.77, tol = 9.737942861426014e-6
Block iter 6 r = 10.23, logl = -118976.23, tol = 4.533204651142322e-6
Block iter 7 r = 10.15, logl = -118975.97, tol = 2.177281199334983e-6
Block iter 8 r = 10.1, logl = -118975.84, tol = 1.1418875958427824e-6
 17.323631 seconds (13.86 M allocations: 2.123 GiB, 3.13% gc time)


-118975.74446589271

In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.757624971928378; true β = 3.757872325600888
estimated variance component 1 = 0.09291945606379969; true variance component 1 = 0.1
estimated variance component 2 = 0.458284104956497; true variance component 2 = 0.5
estimated r = 100.13821818962555; true r = 100


In [14]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.4545002813326036; true β = 1.4552872326068422
estimated variance component 1 = 0.10705454204548824; true variance component 1 = 0.1
estimated variance component 2 = 0.46216360543903984; true variance component 2 = 0.5
estimated r = 10.053594338117408; true r = 10


# Benchmark/profile code

In [191]:
using BenchmarkTools, SpecialFunctions

function update_HΣ_test!(gcm)
    T = Float64
    fill!(gcm.HΣ, 0.0)
    @inbounds for j in 1:gcm.m
        @simd for i in 1:length(gcm.storage_n)
            gcm.hess1[j, i] = gcm.QF[i, j] * gcm.storage_n[i]
            gcm.hess2[j, i] = gcm.TR[i, j] * gcm.storage_n2[i]
        end
    end
    BLAS.gemm!('N', 'T', -T(1), gcm.hess1, gcm.hess1, T(0), gcm.HΣ)
    BLAS.gemm!('N', 'T', T(1), gcm.hess2, gcm.hess2, T(1), gcm.HΣ)
end

update_HΣ_test! (generic function with 1 method)

In [192]:
gc = gcm.data[1]
β = gcm.β
@btime update_HΣ_test!($gcm)

  65.226 μs (0 allocations: 0 bytes)


2×2 Matrix{Float64}:
 23863.7  23863.7
 23863.7  23863.7

# Let's see how to optimally block update 

## 1 iteration each

In [58]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7640471229274333; true β = 3.757872325600888
estimated variance component 1 = 0.010091096783959048; true variance component 1 = 0.1
estimated variance component 2 = 0.010061528161806528; true variance component 2 = 0.5
estimated r = 59.22108970817275; true r = 100


## 5 iteration each

In [45]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.761120679347187; true β = 3.757872325600888
estimated variance component 1 = 0.04200544450800426; true variance component 1 = 0.1
estimated variance component 2 = 0.020655323963911138; true variance component 2 = 0.5
estimated r = 64.2226158070482; true r = 100


## 10 iteration each

In [51]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7577367229152965; true β = 3.757872325600888
estimated variance component 1 = 0.08454686818713725; true variance component 1 = 0.1
estimated variance component 2 = 0.3859156499408988; true variance component 2 = 0.5
estimated r = 96.07852027325903; true r = 100


## 1 IPOPT iter + complete Newton fit

In [65]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.762274167394871; true β = 3.757872325600888
estimated variance component 1 = 0.044468809218469255; true variance component 1 = 0.1
estimated variance component 2 = 0.046354142614646404; true variance component 2 = 0.5
estimated r = 67.38079696512439; true r = 100


## 5 IPOPT iter + complete Newton fit

In [72]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.759537350779348; true β = 3.757872325600888
estimated variance component 1 = 0.05312813376763176; true variance component 1 = 0.1
estimated variance component 2 = 0.07266779547787298; true variance component 2 = 0.5
estimated r = 70.93622181411288; true r = 100


## 10 IPOPT iter + complete Newton fit (this reached max iter count)

In [79]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.757598085312234; true β = 3.757872325600888
estimated variance component 1 = 0.08847643797375318; true variance component 1 = 0.1
estimated variance component 2 = 0.418885780706873; true variance component 2 = 0.5
estimated r = 98.01186301721411; true r = 100


## Copying GLM.jl

In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.456226847936331; true β = 1.4552872326068422
estimated variance component 1 = 0.10133551134062899; true variance component 1 = 0.1
estimated variance component 2 = 0.4371874111348784; true variance component 2 = 0.5
estimated r = 9.69061414063922; true r = 10


In [73]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.758435534322703; true β = 3.757872325600888
estimated variance component 1 = 0.0934850410967605; true variance component 1 = 0.1
estimated variance component 2 = 0.3827355454688751; true variance component 2 = 0.5
estimated r = 94.45211633725886; true r = 100
